In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [15]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Sakthipriya\Desktop\IBM PROJECT\Data Car Resale Value Prediction (3)\Data\autos.csv", header=0, sep=',', encoding='Latin1',)

In [ ]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Sakthipriya\Downloads\Data Car Resale Value Prediction (3)\Data\autos.csv", header=0, sep=',', encoding='Latin1',)

print(df.seller.value_counts())
df[df.seller!='gewerblich']
df=df.drop('seller',1)
print(df.offerType.value_counts())
df[df.offerType!='Gesuch']
df=df.drop('offerType',1)

In [4]:
print(df.shape)
df=df[(df.powerPS >50) & (df.powerPS <900)]
df=df[(df.yearOfRegistration >=1950) & (df.yearOfRegistration <2017)]
print(df.shape)

(371528, 18)
(309171, 18)


In [5]:
df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen','postalCode','dateCreated'],axis='columns',inplace=True)

In [6]:
new_df=df.copy()
new_df=new_df.drop_duplicates(['price','vehicleType','yearOfRegistration','gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType','notRepairedDamage'])

In [7]:
new_df.gearbox.replace(('manuell','automatik'),('manual','automatic'),inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'),inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [8]:
new_df=new_df[(new_df.price >=100)&(new_df.price <=150000)]
new_df['notRepairedDamage'].fillna(value='not-declared',inplace=True)
new_df['fuelType'].fillna(value='not-declared',inplace=True)
new_df['gearbox'].fillna(value='not-declared',inplace=True)
new_df['vehicleType'].fillna(value='not-declared',inplace=True)
new_df['model'].fillna(value='not-declared',inplace=True)

In [9]:
new_df.to_csv("autos_preprocessed.csv")

In [10]:
from sklearn.preprocessing import LabelEncoder
labels=['gearbox','notRepairedDamage','model','brand','fuelType','vehicleType']
mapper={}
for i in labels:
    mapper[i]=LabelEncoder()
    mapper[i].fit(new_df[i])
    tr=mapper[i].transform(new_df[i])
    np.save(str('classes'+i+'.npy'),mapper[i].classes_)
    print(i,":",mapper[i])
    new_df.loc[:,i+'_labels']=pd.Series(tr,index=new_df.index)
labeled = new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration']+[x+"_labels" for x in labels]]
print(labeled.columns)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [11]:
Y=labeled.iloc[:,0].values
X=labeled.iloc[:,1:].values
Y=Y.reshape(-1,1)

In [12]:
from sklearn.model_selection import cross_val_score,train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X,Y,test_size=0.3,random_state=3)